- Récupération du code source après connexion

In [18]:
"""
Enregistrement du code HTML d’une page projet LPB
-------------------------------------------------
• Connexion Selenium (Edge)
• Attente dynamique (cookies + login)
• Sauvegarde du code source dans exports/code_source_connecte.txt
-------------------------------------------------
pip install selenium webdriver-manager beautifulsoup4
"""

import pathlib, time, re
from selenium import webdriver
from selenium.webdriver.edge.service   import Service  as EdgeService
from selenium.webdriver.edge.options   import Options  as EdgeOptions
from selenium.webdriver.common.by      import By
from selenium.webdriver.support.ui     import WebDriverWait
from selenium.webdriver.support        import expected_conditions as EC
from webdriver_manager.microsoft       import EdgeChromiumDriverManager
from bs4                                import BeautifulSoup

# ───────── paramètres utilisateur ─────────
EMAIL        = "romtaug@gmail.com"
PASSWORD     = "CasentleRORO92i*"
PROJECT_URL  = "https://app.lapremierebrique.fr/fr/projects/20"

# dossier & fichier de sortie
EXPORT_DIR   = pathlib.Path("exports")
EXPORT_DIR.mkdir(exist_ok=True)
OUTFILE      = EXPORT_DIR / "code_source_connecte.txt"

# ───────── Selenium : session Edge ─────────
edge_opts = EdgeOptions()
edge_opts.use_chromium = True
edge_opts.add_argument("--window-size=1920,1080")
# edge_opts.add_argument("--headless=new")    # ← décommente si tu veux en headless

driver = webdriver.Edge(
    service = EdgeService(EdgeChromiumDriverManager().install()),
    options = edge_opts
)
wait = WebDriverWait(driver, 10)

# ───────── connexion LPB ─────────
driver.get("https://app.lapremierebrique.fr/fr/users/sign_in")

# accepter cookies (si présent)
try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//button[contains(text(),'Tout autoriser')]"))).click()
except Exception:
    pass         # pas de bandeau cookies

# remplir formulaire
wait.until(EC.presence_of_element_located((By.NAME, "user[email]")))
driver.find_element(By.NAME, "user[email]").send_keys(EMAIL)
driver.find_element(By.NAME, "user[password]").send_keys(PASSWORD)
driver.find_element(By.NAME, "commit").click()

# ───────── page projet ─────────
driver.get(PROJECT_URL)
time.sleep(10)     # marge : laisse le JS injecter dataLayer & co.

# ───────── sauvegarde HTML ────
html = driver.page_source
OUTFILE.write_text(html, encoding="utf-8")
print(f"✅ Code source enregistré dans : {OUTFILE.resolve()}")

driver.quit()

# ───────── (optionnel) aperçu rapide ─────────
soup = BeautifulSoup(html, "html.parser")
title = soup.h1 and soup.h1.get_text(strip=True)
print("Titre détecté :", title or "<aucun>")

✅ Code source enregistré dans : H:\Desktop\Projet LPB\1 - Source - Python Query\exports\code_source_connecte.txt
Titre détecté : Les Jardins du Port


- Récupération des infos du projet

In [1]:
#!/usr/bin/env python3
"""
Scrape LaPremiereBrique projects (IDs 1 → 800)
----------------------------------------------
• Durée détectée même avec espace fine U+202F.
• Log manquant durée → missing_duration.csv
• Export Excel        → exports/projects_1_800.xlsx
"""

from __future__ import annotations
import re, time
from datetime import datetime
from pathlib import Path

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import os
from pathlib import Path

# FIX : Ne pas utiliser __file__ sous Streamlit
cache_dir = Path.cwd() / "wdm_cache"
os.environ["WDM_LOCAL"] = "1"
os.environ["WDM_CACHE"] = str(cache_dir)

# ───────── PARAMÈTRES ─────────
EMAIL, PASSWORD = "romtaug@gmail.com", "CasentleRORO92i*"
ID_START, ID_END = 1, 800
HEADLESS = False
TIMEOUT  = 20

# ───────── DRIVER SELENIUM ────
opts = EdgeOptions()
opts.use_chromium = True
opts.add_argument("--window-size=1920,1080")
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
if HEADLESS:
    opts.add_argument("--headless=new")

driver = webdriver.Edge(
    service=EdgeService(EdgeChromiumDriverManager().install()),
    options=opts,
)
wait = WebDriverWait(driver, TIMEOUT)

# ───────── HELPERS ────────────
text_or = lambda n, d="": n.get_text(" ", strip=True) if n else d
str2eur = lambda s: float(re.sub(r"[^\d,]", "", s).replace(",", ".") or 0)

SPACE_MAP = str.maketrans({" ": " ", "\u202F": " ", "\u2009": " "})
unspace   = lambda s: s.translate(SPACE_MAP)

# ───────── CONSTANTES ─────────
COLS = [
    "id","extraction","url","nom","Ville","Département","Typologie","Entreprise",
    "Notation_LPB","Sûreté","Mode_rémunération","Billet_min","Taux_pub",
    "Intérêt_contractuel","Durée","Collecte","Maximum","Objectif","Restant",
    "Progression","Investisseurs","Ouverture","Remboursement",
    "Clôture_prog","SIREN","Equipe","Présentation","Elements_financiers",
    "Calendrier","Meta_description","Nb_commentaires","URL_replay",
    "Galerie_photos","Image","Statut",
]
blank = lambda pid: {c: (pid if c=="id" else ("introuvable" if c=="extraction" else "")) for c in COLS}

LOG = Path("missing_duration.csv"); LOG.unlink(missing_ok=True)

# ───────── CONNEXION ──────────
print("🔑 Connexion…")
driver.get("https://app.lapremierebrique.fr/fr/users/sign_in")
try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH,"//button[contains(text(),'Tout autoriser')]"))).click()
except Exception: pass
wait.until(EC.presence_of_element_located((By.NAME,"user[email]")))
driver.find_element(By.NAME,"user[email]").send_keys(EMAIL)
driver.find_element(By.NAME,"user[password]").send_keys(PASSWORD)
driver.find_element(By.NAME,"commit").click()
print("✅ Connecté")

# ───────── FONCTIONS ──────────
def recap_stats(s:BeautifulSoup)->dict[str,str]:
    d={}
    for td in s.select("table tbody tr td.bg-body-secondary"):
        grey=text_or(td.find("span",class_="text-dark-grey")).lower()
        bold=text_or(td.find("span",class_="fw-bold"))
        if grey: d[grey]=bold
    return d

def data_layer(s:BeautifulSoup)->tuple[str,str]:
    typ, tgt = "",""
    for sc in s.find_all("script"):
        t=sc.text
        if not typ and (m:=re.search(r'"typology"\s*:\s*"([^"]+)"',t)): typ=m.group(1)
        if not tgt and (m:=re.search(r'"targeted_amount"\s*:\s*"([^"]+)"',t)): tgt=m.group(1)
        if typ and tgt: break
    return typ,tgt

def section(s:BeautifulSoup, rx:str)->str:
    h=s.find("h3",string=re.compile(rx,re.I)); out=[]
    while h and (h:=h.find_next_sibling()):
        if h.name=="h3": break
        out.append(text_or(h))
    return " ".join(out).strip()

ENT_REGEX = re.compile(r"\b(SAS|SCI|SARL|SA)\s+[A-Z0-9À-Ÿ'’\- ]{3,}")

def parse(html:str):
    soup=BeautifulSoup(html,"html.parser")
    og=soup.select_one('[property="og:url"]')
    if not og or not (mid:=re.search(r"/projects/(\d+)$",og["content"])): return None
    pid=int(mid.group(1)); url=og["content"]

    title=text_or(soup.h1)
    subtitle=text_or(soup.select_one("h2.h6"))

    city=dep=""
    if (m:=re.search(r"(.+?)\s*\((\d{2})\)",subtitle)): city,dep=m.group(1).strip(),m.group(2)

    stats=recap_stats(soup)
    collected=stats.get("collectés",stats.get("collecté",""))
    maximum  =stats.get("maximum","")
    investors=stats.get("investisseurs",stats.get("investisseur",""))

    duree=interet=""
    for g,b in stats.items():
        if (m:=re.search(r"\b(\d+)\s*mois\b",unspace(g))):
            duree=f"{m.group(1)} mois"; interet=b; break
    if not duree and (m:=re.search(r"durée[^0-9]*(\d+)\s*mois",unspace(subtitle),re.I)):
        duree=f"{m.group(1)} mois"

    bar=soup.select_one(".progressbar .bar[style]")
    progress=f"{float(re.search(r'([\d.]+)',bar['style']).group()):.0f} %" if bar else ""

    typ_badge=text_or(soup.select_one("span.pill-type"))
    typ_json,obj_json=data_layer(soup)
    typology=typ_badge or typ_json or (soup.select_one("[data-typology]") and soup.select_one("[data-typology]")["data-typology"]) or ""
    objective=obj_json

    presentation=section(soup,"Présentation")
    entreprise=""
    if (m:=ENT_REGEX.search(presentation)): entreprise=m.group(0).strip()

    ticket_min=taux_pub=""
    if (m:=re.search(r"Investissez dès ([\d ]+)€",subtitle)): ticket_min=f"{m.group(1).replace(' ','')} €"
    if (m:=re.search(r"au taux de ([\d,.% ]+)",subtitle)):    taux_pub=m.group(1).strip()

    note_lpb=text_or(soup.select_one("span.pill"))

    surete=""
    if (h:=soup.find("h3",string=re.compile("sûretés",re.I))):
        if (li:=h.find_next("li",class_="border-top")): surete=text_or(li)

    mode_rem=text_or(soup.find("li",string=re.compile("Mode de rémunération",re.I)))
    elements_fin=section(soup,"Eléments financiers")
    calendrier   =section(soup,"Calendrier prévisionnel")

    equipe=""
    if (h:=soup.find("h3",string=re.compile("Équipe projet",re.I))) and (ul:=h.find_next("ul")):
        equipe="; ".join(text_or(li) for li in ul.find_all("li"))

    siren=""
    if (m:=re.search(r"\b\d{9}\b",elements_fin or presentation)): siren=m.group()

    closing=""
    if (m:=re.search(r"Clôture[^:]*:\s*([^.;]+)",elements_fin,re.I)): closing=m.group(1).strip()

    ouverture=remboursement=""
    if (a:=soup.select_one('a[href*="calendar/render"][href*="dates="]')) and (m:=re.search(r"dates=(\d{8}T\d{6})",a["href"])):
        ouverture=datetime.strptime(m.group(1),"%Y%m%dT%H%M%S").strftime("%d/%m/%Y %H:%M")
    if (t:=soup.find("time",datetime=True)): remboursement=t["datetime"]

    img_url=status_url=""
    if (cover:=soup.select_one("#projectCover")):
        if (m:=re.search(r"url\((.*?)\)",cover.get("style",""))): img_url=m.group(1)
        if (st:=cover.select_one("img.project-status-show")):      status_url=st["src"]

    meta_desc=text_or(soup.select_one('[name="description"],[property="og:description"]'))
    nb_com   =text_or(soup.select_one("#comments span.count"))
    replay_if=soup.select_one('iframe[src*="youtube.com/embed"]')
    replay   =replay_if["src"] if replay_if else ""
    gallery  =", ".join(img["src"] for img in soup.select("#projectGallery img[src]"))

    restant=""
    if collected and objective:
        restant=f"{str2eur(objective)-str2eur(collected):,.0f} €".replace(",", " ")

    if not duree:
        with LOG.open("a",encoding="utf-8") as f: f.write(f"{pid};{url}\n")

    return {
        "id":pid,"extraction":"trouvé","url":url,"nom":title,
        "Ville":city,"Département":dep,"Typologie":typology,"Entreprise":entreprise,
        "Notation_LPB":note_lpb,"Sûreté":surete,"Mode_rémunération":mode_rem,
        "Billet_min":ticket_min,"Taux_pub":taux_pub,"Intérêt_contractuel":interet,
        "Durée":duree,"Collecte":collected,"Maximum":maximum,"Objectif":objective,
        "Restant":restant,"Progression":progress,"Investisseurs":investors,
        "Ouverture":ouverture,"Remboursement":remboursement,"Clôture_prog":closing,
        "SIREN":siren,"Equipe":equipe,"Présentation":presentation,
        "Elements_financiers":elements_fin,"Calendrier":calendrier,"Meta_description":meta_desc,
        "Nb_commentaires":nb_com,"URL_replay":replay,"Galerie_photos":gallery,
        "Image":img_url,"Statut":status_url,
    }

# ───────── SCRAPE LOOP ───────
print("🚀 Démarrage du scrape…")
rows=[]
for pid in range(ID_START,ID_END+1):
    driver.get(f"https://app.lapremierebrique.fr/fr/projects/{pid}")
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1"))); time.sleep(2)
    except Exception: pass
    rows.append(parse(driver.page_source) or blank(pid))
    print(f"{pid:>4} – {'OK' if rows[-1]['extraction']=='trouvé' else 'introuvable'}")

# ───────── EXPORT EXCEL ──────
Path("exports").mkdir(exist_ok=True)
out=Path("exports/projects_1_800.xlsx")
pd.DataFrame(rows,columns=COLS).to_excel(out,index=False)
print("✅ Export :",out.resolve())
if LOG.exists(): print("⚠️ Durées manquantes listées dans",LOG.resolve())
driver.quit(); print("🎉 Terminé")

🔑 Connexion…
✅ Connecté
🚀 Démarrage du scrape…
   1 – OK



KeyboardInterrupt

